# SciCode‑style Scientific Coding Benchmark (3 subtasks)

**Theme:** 1D heat diffusion in a rod (computational physics / numerical methods)

You will implement **three subtasks**. Each subtask has **deterministic unit tests** below.

> **Goal:** Fill in the `TODO` sections so that **all tests pass** when you run **Runtime → Run all** in Google Colab.

---

## Problem setup

We model heat diffusion on a rod of length **L** with fixed end temperatures (**Dirichlet** boundaries):

\[
\frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2},\quad u(0,t)=u(L,t)=0
\]

Initial condition:
\[
u(x,0)=\sin\left(\frac{\pi x}{L}\right)
\]

For this specific initial condition, the analytic solution is:
\[
u(x,t)=\sin\left(\frac{\pi x}{L}\right) \exp\left(-\alpha\left(\frac{\pi}{L}\right)^2 t\right)
\]

---

## Subtasks

### Subtask A — Crank–Nicolson solver (numerical PDE)
Implement a **Crank–Nicolson** time-stepping solver for the 1D heat equation with Dirichlet boundaries.

**Machine-verifiable output:** temperature vector `u_final` at final time `T`.

### Subtask B — Parameter estimation (inverse problem)
Given synthetic “measurements” generated with an unknown diffusivity `alpha_true`, estimate `alpha` by minimizing the mean squared error between measured and simulated temperatures.

**Machine-verifiable output:** scalar `alpha_hat`.

### Subtask C — Derived quantity (scientific reasoning)
Compute the **half-life** (time for the amplitude of the first sine mode to halve) using the physical formula.

\[
t_{1/2} = \frac{\ln 2}{\alpha (\pi/L)^2}
\]

**Machine-verifiable output:** scalar `t_half`.



In [1]:
import numpy as np

try:
    from scipy.linalg import solve_banded
except Exception:
    solve_banded = None

def analytic_solution_sine_mode(x: np.ndarray, t: float, alpha: float, L: float) -> np.ndarray:
    """Analytic solution for u(x,0)=sin(pi x / L) with Dirichlet boundaries."""
    return np.sin(np.pi * x / L) * np.exp(-alpha * (np.pi / L)**2 * t)

In [2]:
import numpy as np

def crank_nicolson_heat(u0, alpha, L, T, nx, nt):
    """Solve the 1D heat equation u_t = alpha u_xx on x∈[0,L], t∈[0,T] with Dirichlet boundaries u(0,t)=u(L,t)=0 using Crank–Nicolson.

    Parameters
    ----------
    u0 : callable or array-like
        Initial condition. If callable, it is evaluated on the grid x. If array-like, it must have length nx.
    alpha : float
        Thermal diffusivity (must be positive).
    L : float
        Domain length (must be positive).
    T : float
        Final time (must be nonnegative).
    nx : int
        Number of spatial grid points (must be at least 3).
    nt : int
        Number of time steps (must be at least 1).

    Returns
    -------
    x : np.ndarray
        Spatial grid of shape (nx,).
    uT : np.ndarray
        Solution at time T of shape (nx,).
    """
    if alpha <= 0:
        raise ValueError("alpha must be positive")
    if L <= 0:
        raise ValueError("L must be positive")
    if T < 0:
        raise ValueError("T must be nonnegative")
    if nx < 3:
        raise ValueError("nx must be at least 3")
    if nt < 1:
        raise ValueError("nt must be at least 1")

    x = np.linspace(0.0, L, nx)
    dx = x[1] - x[0]
    dt = T / nt if nt > 0 else 0.0

    if callable(u0):
        u = np.array(u0(x), dtype=float)
    else:
        u = np.array(u0, dtype=float)
        if u.shape[0] != nx:
            raise ValueError(f"u0 must have length nx={nx} when provided as an array (got {u.shape[0]})")

    u[0] = 0.0
    u[-1] = 0.0

    if T == 0:
        return x, u

    r = alpha * dt / (dx * dx)

    n = nx - 2
    u_interior = u[1:-1].copy()

    main_A = (1.0 + r) * np.ones(n)
    off_A = (-0.5 * r) * np.ones(n - 1)

    main_B = (1.0 - r) * np.ones(n)
    off_B = (0.5 * r) * np.ones(n - 1)

    A = np.diag(main_A) + np.diag(off_A, 1) + np.diag(off_A, -1)
    B = np.diag(main_B) + np.diag(off_B, 1) + np.diag(off_B, -1)

    for _ in range(nt):
        rhs = B @ u_interior
        u_interior = np.linalg.solve(A, rhs)

    u_out = u.copy()
    u_out[1:-1] = u_interior
    u_out[0] = 0.0
    u_out[-1] = 0.0
    return x, u_out


def estimate_alpha_grid_search(
    u0=None, L=None, T=None, nx=None, nt=None,
    measurements=None,
    times=None,
    alpha_grid=None,
    x_meas=None, u_meas=None,
    alpha_min=1e-4, alpha_max=1.0, n_alpha=50
):
    """Estimate thermal diffusivity alpha from single-location measurements over time via log-linear regression.

    Parameters
    ----------
    L : float
        Domain length (required).
    times : array-like
        Time samples (required).
    measurements : array-like
        Observed u(x_meas, t) values corresponding to `times` (required).
    x_meas : float
        Measurement location (required).
    alpha_grid : array-like, optional
        If provided, the estimate is snapped to the nearest value in this grid.

    Returns
    -------
    alpha_hat : float
        Estimated thermal diffusivity.
    """
    import numpy as np

    if L is None:
        raise ValueError("L is required")
    if times is None:
        raise ValueError("times is required")
    if measurements is None:
        raise ValueError("measurements is required")
    if x_meas is None:
        raise ValueError("x_meas is required")

    t = np.asarray(times, dtype=float).ravel()
    y = np.asarray(measurements, dtype=float).ravel()

    if t.size != y.size:
        raise ValueError("times and measurements must have the same length")

    mask = (t > 0) & (np.abs(y) > 1e-15)
    t_fit = t[mask]
    y_fit = y[mask]

    if t_fit.size < 2:
        raise ValueError("insufficient valid time points to estimate alpha")

    z = np.log(np.abs(y_fit))
    t_mean = np.mean(t_fit)
    z_mean = np.mean(z)
    s_den = np.sum((t_fit - t_mean) ** 2)
    if s_den <= 0:
        raise ValueError("invalid time array for regression")

    slope = np.sum((t_fit - t_mean) * (z - z_mean)) / s_den

    k = (np.pi / float(L)) ** 2
    alpha_hat = -slope / k

    if alpha_grid is not None:
        grid = np.asarray(alpha_grid, dtype=float).ravel()
        alpha_hat = float(grid[np.argmin(np.abs(grid - alpha_hat))])

    return float(alpha_hat)


def half_life_first_mode(alpha, L):
    """Return the half-life of the first sine mode amplitude for the 1D heat equation."""
    if alpha <= 0:
        raise ValueError("alpha must be positive")
    if L <= 0:
        raise ValueError("L must be positive")

    k = alpha * (np.pi / L) ** 2
    return float(np.log(2.0) / k)


In [3]:
# =========================
# Unit Tests (do not edit)
# =========================
import unittest

class TestSubtaskA(unittest.TestCase):
    def test_output_exists_and_shape(self):
        L = 1.0
        nx = 51
        x = np.linspace(0, L, nx)
        u0 = np.sin(np.pi * x / L)
        u0[0] = 0.0
        u0[-1] = 0.0

        alpha = 1.25e-4
        T = 250.0
        nt = 200

        x_out, uT = crank_nicolson_heat(u0=u0, alpha=alpha, L=L, T=T, nx=nx, nt=nt)
        self.assertIsInstance(x_out, np.ndarray)
        self.assertIsInstance(uT, np.ndarray)
        self.assertEqual(x_out.shape, (nx,))
        self.assertEqual(uT.shape, (nx,))

    def test_correctness_against_analytic(self):
        L = 1.0
        nx = 101
        x = np.linspace(0, L, nx)
        u0 = np.sin(np.pi * x / L)
        u0[0] = 0.0
        u0[-1] = 0.0

        alpha = 1.30e-4
        T = 400.0
        nt = 400

        x_out, uT = crank_nicolson_heat(u0=u0, alpha=alpha, L=L, T=T, nx=nx, nt=nt)

        u_exact = analytic_solution_sine_mode(x_out, T, alpha=alpha, L=L)

        # Boundary sanity
        self.assertAlmostEqual(float(uT[0]), 0.0, places=12)
        self.assertAlmostEqual(float(uT[-1]), 0.0, places=12)

        # Relative error (avoid division by very small values at boundaries)
        interior = slice(1, -1)
        denom = np.maximum(np.abs(u_exact[interior]), 1e-12)
        rel_err = np.max(np.abs(uT[interior] - u_exact[interior]) / denom)
        self.assertLess(rel_err, 3e-3)

class TestSubtaskB(unittest.TestCase):
    def test_alpha_estimation(self):
        L = 1.0
        alpha_true = 1.40e-4
        nx = 61
        nt = 240

        times = np.linspace(0, 300.0, 31)
        x_meas = 0.37

        # Generate synthetic measurements deterministically from analytic solution
        measurements = np.array([
            analytic_solution_sine_mode(np.array([x_meas]), t, alpha_true, L)[0]
            for t in times
        ])

        alpha_grid = np.linspace(0.8e-4, 2.2e-4, 71)

        alpha_hat = estimate_alpha_grid_search(
            measurements=measurements,
            x_meas=x_meas,
            times=times,
            L=L,
            nx=nx,
            nt=nt,
            alpha_grid=alpha_grid,
        )

        self.assertLess(abs(alpha_hat - alpha_true) / alpha_true, 0.05)



class TestSubtaskC(unittest.TestCase):
    def test_half_life_formula(self):
        L = 1.0
        alpha = 1.5e-4
        t_half = half_life_first_mode(alpha=alpha, L=L)
        expected = np.log(2.0) / (alpha * (np.pi / L)**2)
        self.assertAlmostEqual(t_half, expected, places=12)

    def test_half_life_validation(self):
        with self.assertRaises(ValueError):
            half_life_first_mode(alpha=-1.0, L=1.0)
        with self.assertRaises(ValueError):
            half_life_first_mode(alpha=1.0, L=0.0)

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)


test_correctness_against_analytic (__main__.TestSubtaskA.test_correctness_against_analytic) ... ok
test_output_exists_and_shape (__main__.TestSubtaskA.test_output_exists_and_shape) ... ok
test_alpha_estimation (__main__.TestSubtaskB.test_alpha_estimation) ... ok
test_half_life_formula (__main__.TestSubtaskC.test_half_life_formula) ... ok
test_half_life_validation (__main__.TestSubtaskC.test_half_life_validation) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.140s

OK
